# Structured Outputs

LLMs regurgitate out text and that is great for so many applications. But in order to build strong, robust systems and applications, we need to make sense of the chaos sometimes by receiving a pre-determined structured output everytime an LLM is called.

## As always, libraries first!

In [39]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")

## Introducing the Pydantic library

In [40]:
# classes
from pydantic import BaseModel

class ProductReview(BaseModel):
    product: str
    review: str

class ReviewResponse(BaseModel):
    response: str
    response_time: str

class ReviewEvaluation(BaseModel):
    passed: bool
    feedback: str

## Calling OpenAI to generate support tickets

In [41]:
# client
client = OpenAI()

In [42]:
# messages list
user_message = "I want you to generate a product review for Nvidia RTX 5070 12GB GPU."
user_message += "The review should concise and include pros and cons of the product."

messages = [{"role": "user", "content": user_message}]

In [43]:
# normal review
response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)

normal_review = response.choices[0].message.content
display(Markdown(f"### Normal Review:\n{normal_review}"))

### Normal Review:
**Nvidia RTX 5070 12GB GPU Review**

The Nvidia RTX 5070 12GB delivers solid gaming performance with enhanced ray tracing capabilities and ample VRAM for demanding titles. It’s a great choice for gamers seeking high-quality visuals without breaking the bank.

**Pros:**
- Strong 12GB GDDR6 memory ideal for modern games and creative workloads
- Improved ray tracing and DLSS support for enhanced visuals
- Solid performance across current AAA titles
- Relatively efficient power consumption

**Cons:**
- Slightly higher price compared to previous generation mid-range cards
- Limited availability may affect purchase options
- Not a significant leap over the previous generation for some users

Overall, the RTX 5070 offers good value for gamers and creatives looking for a balanced high-performance GPU.

In [45]:
# structured review
structured_review = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ProductReview
)

structured_review = structured_review.choices[0].message.parsed
display(Markdown(f"### Structured Review:\n{structured_review}"))

### Structured Review:
product='Nvidia RTX 5070 12GB GPU' review='The Nvidia RTX 5070 12GB GPU offers solid performance for gaming and creative workloads, thanks to its ample VRAM and advanced architecture. It handles modern titles at high settings with ease and supports ray tracing for enhanced visuals. However, it can be quite power-hungry and runs hot under load, necessitating effective cooling solutions. Its premium price may also be a consideration for budget-conscious users.'

In [46]:
structured_review.product

'Nvidia RTX 5070 12GB GPU'

In [47]:
structured_review.review

'The Nvidia RTX 5070 12GB GPU offers solid performance for gaming and creative workloads, thanks to its ample VRAM and advanced architecture. It handles modern titles at high settings with ease and supports ray tracing for enhanced visuals. However, it can be quite power-hungry and runs hot under load, necessitating effective cooling solutions. Its premium price may also be a consideration for budget-conscious users.'

## Responding to the review

In [48]:
# messages list
message = "You are to propose a response for the following product review. \n\n"
message += f"Product: {structured_review.product}\n"
message += f"Review: {structured_review.review}\n"

messages = [{"role": "user", "content": message}]

In [ ]:
# structured response
review_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ReviewResponse
)

review_response = review_response.choices[0].message.parsed
display(Markdown(f"### Response:\n{review_response.response}"))
display(Markdown(f"### Response Time:\n{review_response.response_time}"))

### Response:
Thank you for sharing your insights on the Nvidia RTX 5070 12GB GPU. It's great to hear that it delivers strong performance in gaming and creative tasks, with features like ample VRAM and ray tracing support enhancing the user experience. We appreciate your notes on its power consumption and heat generation, highlighting the importance of proper cooling solutions. Your mention of its price point is also valuable for potential buyers considering their budget. Your review provides a balanced perspective that will help others make informed decisions.

### Response Time:
2023-10-05T12:00:00Z

## Lets evaluate our response

In [50]:
# messages list
message = "You are to evaluate the response for the following product review. "
message += "You will determine if the proposed response is appropriate for the product review and priority level. "
message += f"Product: {structured_review.product}\n"
message += f"Proposed Response {review_response.response}\n\n"

messages = [{"role": "user", "content": message}]

In [51]:
messages

[{'role': 'user',
  'content': "You are to evaluate the response for the following product review. You will determine if the proposed response is appropriate for the product review and priority level. Product: Nvidia RTX 5070 12GB GPU\nProposed Response Thank you for sharing your insights on the Nvidia RTX 5070 12GB GPU. It's great to hear that it delivers strong performance in gaming and creative tasks, with features like ample VRAM and ray tracing support enhancing the user experience. We appreciate your notes on its power consumption and heat generation, highlighting the importance of proper cooling solutions. Your mention of its price point is also valuable for potential buyers considering their budget. Your review provides a balanced perspective that will help others make informed decisions.\n\n"}]

In [52]:
# evaluate response
evaluator_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ReviewEvaluation
)

evaluator_response = evaluator_response.choices[0].message.parsed
display(Markdown(f"### Passed:\n{evaluator_response.passed}"))
display(Markdown(f"### Feedback:\n{evaluator_response.feedback}"))

### Passed:
True

### Feedback:
The response is appropriate and well-balanced, acknowledging the user's insights and emphasizing key aspects of the Nvidia RTX 5070 12GB GPU. It effectively addresses the review's points and provides useful information for potential buyers.